Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Inference PyTorch Bert Model with ONNX Runtime on GPU

In this tutorial, you'll be introduced to how to load a Bert model from PyTorch, convert it to ONNX, and inference it for high performance using ONNX Runtime and NVIDIA GPU. In the following sections, we are going to use the Bert model trained with Stanford Question Answering Dataset (SQuAD) dataset as an example. Bert SQuAD model is used in question answering scenarios, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

This notebook is for GPU inference. For CPU inference, please look at another notebook [Inference PyTorch Bert Model with ONNX Runtime on CPU](PyTorch_Bert-Squad_OnnxRuntime_CPU.ipynb).

## 0. Prerequisites ##
It requires your machine to have a GPU, and a python environment with [PyTorch](https://pytorch.org/) and [OnnxRuntime](https://microsoft.github.io/onnxruntime/) installed before running this notebook.

#### GPU Environment Setup using AnaConda

First, we install [AnaConda](https://www.anaconda.com/distribution/) in a target machine and open an AnaConda prompt window when it is done. Then run the following commands to create a conda environment. This notebook is tested with PyTorch 1.4 and OnnxRuntime 1.2.0.

```console
conda create -n gpu_env python=3.6
conda activate gpu_env
conda install pytorch torchvision cudatoolkit=10.1 -c pytorch
pip install onnxruntime-gpu
pip install transformers==2.5.1
pip install wget psutil onnx pytz pandas py-cpuinfo py3nvml netron
conda install jupyter
jupyter notebook
```

Onnxruntime-gpu need specified version of CUDA and cuDNN. You can find the corresponding version in [release note](https://github.com/microsoft/onnxruntime/releases). If the version is different from above cudatoolkit version, you have to install them separately, and add their bin directories to PATH environment variable (See [CUDA and cuDNN Path](#CUDA-and-cuDNN-Path) below).

## 1. Load Pretrained Bert model ##

We begin by downloading the SQuAD data file and store them in the specified location. 

In [1]:
import os

cache_dir = "./squad"
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

predict_file_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"
predict_file = os.path.join(cache_dir, "dev-v1.1.json")
if not os.path.exists(predict_file):
    import wget
    print("Start downloading predict file.")
    wget.download(predict_file_url, predict_file)
    print("Predict file downloaded.")

Let's first define some constant variables.

In [2]:
# Whether allow overwriting existing ONNX model and download the latest script from GitHub
enable_overwrite = True

# Total samples to inference, so that we can get average latency
total_samples = 1000

# ONNX opset version: 10 or 11
opset_version=11

Specify some model configuration variables.

In [3]:
# For fine-tuned large model, the model name is "bert-large-uncased-whole-word-masking-finetuned-squad". Here we use bert-base for demo.
model_name_or_path = "bert-base-cased"
max_seq_length = 128
doc_stride = 128
max_query_length = 64

Start to load model from pretrained. This step could take a few minutes. 

In [4]:
# The following code is adapted from HuggingFace transformers
# https://github.com/huggingface/transformers/blob/master/examples/run_squad.py

from transformers import (BertConfig, BertForQuestionAnswering, BertTokenizer)

# Load pretrained model and tokenizer
config_class, model_class, tokenizer_class = (BertConfig, BertForQuestionAnswering, BertTokenizer)
config = config_class.from_pretrained(model_name_or_path, cache_dir=cache_dir)
tokenizer = tokenizer_class.from_pretrained(model_name_or_path, do_lower_case=True, cache_dir=cache_dir)
model = model_class.from_pretrained(model_name_or_path,
                                    from_tf=False,
                                    config=config,
                                    cache_dir=cache_dir)
# load some examples
from transformers.data.processors.squad import SquadV1Processor

processor = SquadV1Processor()
examples = processor.get_dev_examples(None, filename=predict_file)

from transformers import squad_convert_examples_to_features
features, dataset = squad_convert_examples_to_features( 
            examples=examples[:total_samples], # convert enough examples for this notebook
            tokenizer=tokenizer,
            max_seq_length=max_seq_length,
            doc_stride=doc_stride,
            max_query_length=max_query_length,
            is_training=False,
            return_dataset='pt'
        )

add example index and unique id: 100%|█████████████████████████████████████████| 1000/1000 [00:00<00:00, 500274.81it/s]


## 2. Export the loaded model ##
Once the model is loaded, we can export the loaded PyTorch model to ONNX.

In [5]:
output_dir = "./onnx"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)   
export_model_path = os.path.join(output_dir, 'bert-base-cased-squad_opset{}.onnx'.format(opset_version))

import torch
use_gpu = torch.cuda.is_available()
device = torch.device("cuda" if use_gpu else "cpu")

# Get the first example data to run the model and export it to ONNX
data = dataset[0]
inputs = {
    'input_ids':      data[0].to(device).reshape(1, max_seq_length),
    'attention_mask': data[1].to(device).reshape(1, max_seq_length),
    'token_type_ids': data[2].to(device).reshape(1, max_seq_length)
}

# Set model to inference mode, which is required before exporting the model because some operators behave differently in 
# inference and training mode.
model.eval()
model.to(device)

if enable_overwrite or not os.path.exists(export_model_path):
    with torch.no_grad():
        symbolic_names = {0: 'batch_size', 1: 'max_seq_len'}
        torch.onnx.export(model,                                            # model being run
                          args=tuple(inputs.values()),                      # model input (or a tuple for multiple inputs)
                          f=export_model_path,                              # where to save the model (can be a file or file-like object)
                          opset_version=opset_version,                      # the ONNX version to export the model to
                          do_constant_folding=True,                         # whether to execute constant folding for optimization
                          input_names=['input_ids',                         # the model's input names
                                       'input_mask', 
                                       'segment_ids'],
                          output_names=['start', 'end'],                    # the model's output names
                          dynamic_axes={'input_ids': symbolic_names,        # variable length axes
                                        'input_mask' : symbolic_names,
                                        'segment_ids' : symbolic_names,
                                        'start' : symbolic_names,
                                        'end' : symbolic_names})
        print("Model exported at ", export_model_path)

Model exported at  ./onnx\bert-base-cased-squad_opset11.onnx


## 3. PyTorch Inference ##
Use PyTorch to evaluate an example input for comparison purpose.

In [6]:
import time

# Measure the latency. It is not accurate using Jupyter Notebook, it is recommended to use standalone python script.
latency = []
with torch.no_grad():
    for i in range(total_samples):
        data = dataset[i]
        inputs = {
            'input_ids':      data[0].to(device).reshape(1, max_seq_length),
            'attention_mask': data[1].to(device).reshape(1, max_seq_length),
            'token_type_ids': data[2].to(device).reshape(1, max_seq_length)
        }
        start = time.time()
        outputs = model(**inputs)
        latency.append(time.time() - start)
print("PyTorch {} Inference time = {} ms".format(device.type, format(sum(latency) * 1000 / len(latency), '.2f')))

PyTorch cuda Inference time = 16.54 ms


## 4. Inference ONNX Model with ONNX Runtime ##

### CUDA and cuDNN Path
onnxruntime-gpu has dependency on [CUDA](https://developer.nvidia.com/cuda-downloads) and [cuDNN](https://developer.nvidia.com/cudnn):

* [onnxruntime-gpu v1.2.0](https://github.com/microsoft/onnxruntime/releases/tag/v1.2.0) requires CUDA Runtime 10.1.243 and CUDNN 7.6.5.32.
* [onnxruntime-gpu v1.0.0](https://github.com/microsoft/onnxruntime/releases/tag/v1.0.0) ~ v1.1.2 requires CUDA Runtime 10.0 and CUDNN 7.6.

During installing PyTorch 1.4, we installed cudatoolkit 10.1.243 in this conda environment. That shall be good for onnxruntime-gpu 1.2.0 in Jupyter Notebook.

If you use onnxruntime-gpu 1.0.0 ~ 1.1.2, you will have to install CUDA and CUDNN, then add them to path like the following:

In [7]:
# Change to True when onnxruntime (like onnxruntime-gpu 1.0.0 ~ 1.1.2) cannot be imported.
add_cuda_path = False

if add_cuda_path:
    # Add path of CUDA 10.0 and CUDNN 7.6 for onnxruntime-gpu 1.0.0 ~ 1.1.2
    cuda_dir = 'D:/NVidia/CUDA/v10.0/bin'
    cudnn_dir = 'D:/NVidia/CUDA/v10.0/bin'
    if not (os.path.exists(cuda_dir) and os.path.exists(cudnn_dir)):
        raise ValueError("Please specify correct path for CUDA and cuDNN. Otherwise onnxruntime cannot be imported.")
    else:
        if cuda_dir == cudnn_dir:
            os.environ["PATH"] = cuda_dir + ';' + os.environ["PATH"]
        else:
            os.environ["PATH"] = cuda_dir + ';' + cudnn_dir + ';' + os.environ["PATH"]

### OpenMP Environment Variable

OpenMP environment variables are optional for GPU inference of standard Bert model. It has little performance impact on Bert model since most nodes are executed in GPU. 

You can find the best setting based on [Performance Test Tool](#Performance-Test-Tool) result in later part of this notebook.

**Attention: Setting environment variables shall be done before importing onnxruntime**. Otherwise, they might not take effect.

In [8]:
# Optional. You can change them according to Performance Test Tool result.
#os.environ["OMP_NUM_THREADS"] = '1'
#os.environ["OMP_WAIT_POLICY"] = 'PASSIVE'

Now we are ready to inference the model with ONNX Runtime.

In [9]:
import psutil
import onnxruntime
import numpy

assert 'CUDAExecutionProvider' in onnxruntime.get_available_providers()
device_name = 'gpu'

sess_options = onnxruntime.SessionOptions()

# Optional: store the optimized graph and view it using Netron to verify that model is fully optimized.
# Note that this will increase session creation time so enable it for debugging only.
sess_options.optimized_model_filepath = os.path.join(output_dir, "optimized_model_{}.onnx".format(device_name))

# Please change the value according to best setting in Performance Test Tool result.
sess_options.intra_op_num_threads=psutil.cpu_count(logical=True)

session = onnxruntime.InferenceSession(export_model_path, sess_options)

latency = []
for i in range(total_samples):
    data = dataset[i]
    # Use contiguous array as input might improve performance
    ort_inputs = {
        'input_ids':  numpy.ascontiguousarray(data[0].cpu().reshape(1, max_seq_length).numpy()),
        'input_mask': numpy.ascontiguousarray(data[1].cpu().reshape(1, max_seq_length).numpy()),
        'segment_ids': numpy.ascontiguousarray(data[2].cpu().reshape(1, max_seq_length).numpy())
    }
    start = time.time()
    ort_outputs = session.run(None, ort_inputs)
    latency.append(time.time() - start)
    
print("OnnxRuntime {} Inference time = {} ms".format(device_name, format(sum(latency) * 1000 / len(latency), '.2f')))

OnnxRuntime gpu Inference time = 3.72 ms


We can compare the output of PyTorch and ONNX Runtime. We can see some results are not close. It is because ONNX Runtime uses some approximation in CUDA optimization. Based on our evaluation on SQuAD data set, F1 score is on par for models before and after optimization.

In [10]:
print("***** Verifying correctness *****")
for i in range(2):    
    print('PyTorch and ONNX Runtime output {} are close:'.format(i), numpy.allclose(ort_outputs[i], outputs[i].cpu(), rtol=1e-02, atol=1e-02))
    diff = ort_outputs[i] - outputs[i].cpu().numpy()
    max_diff = numpy.max(numpy.abs(diff))
    avg_diff = numpy.average(numpy.abs(diff))
    print(f'maximum_diff={max_diff} average_diff={avg_diff}')

***** Verifying correctness *****
PyTorch and ONNX Runtime output 0 are close: True
maximum_diff=0.005700558423995972 average_diff=0.0005014391499571502
PyTorch and ONNX Runtime output 1 are close: True
maximum_diff=0.0023995935916900635 average_diff=0.00045171938836574554


### Inference with Actual Sequence Length
Note that ONNX model is exported using dynamic length axis. It is recommended to use actual sequence input without padding instead of fixed length input for best performance. Let's see how it can be applied to this model.

From an example input below, we can see zero padding at the end of each sequence.

In [11]:
# An example input (we can see padding). From attention_mask, we can deduce the actual length.
inputs

{'input_ids': tensor([[  101,  1293,  1242,  2557,  1127,  1226,  1104,  1103,  3613, 16429,
           5235,   136,   102,  3613, 16429,  5988,   170,   107,  1353,  1671,
           1992,  1342,   107,  5235,   117,  1107,  1134,  1473,  3683,  3538,
           1125,   170,  1476,   118,  1248,  2595,  4086,  1714,  1104,  2965,
          15897,  1104,  3613, 16429,   119,  1473,  3683,  3538,  3222,  1149,
           2551,  1168, 23759,  1116,  1121,  1506,  1103, 10280,  2231,  1111,
           1103,  1714, 16355,   119,   102,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,    

The original sequence length is 128. After removing paddings, the sequence length is reduced. Input with smaller sequence length need less computation, thus we can see there is improvement on inference latency. 

In [12]:
import statistics

latency = []
lengths = []
for i in range(total_samples):
    data = dataset[i]
    # Instead of using fixed length (128), we can use actual sequence length (less than 128), which helps to get better performance.
    actual_sequence_length = sum(data[1].numpy())
    lengths.append(actual_sequence_length)
    opt_inputs = {
        'input_ids':  data[0].numpy()[:actual_sequence_length].reshape(1, actual_sequence_length),
        'input_mask': data[1].numpy()[:actual_sequence_length].reshape(1, actual_sequence_length),
        'segment_ids': data[2].numpy()[:actual_sequence_length].reshape(1, actual_sequence_length)
    }
    start = time.time()
    opt_outputs = session.run(None, opt_inputs)
    latency.append(time.time() - start)
print("Average length", statistics.mean(lengths))
print("OnnxRuntime {} Inference time with actual sequence length = {} ms".format(device_name, format(sum(latency) * 1000 / len(latency), '.2f')))

Average length 101
OnnxRuntime gpu Inference time with actual sequence length = 3.44 ms


Let's compare the output and see whether the results are close.

**Note**: Need end-to-end evaluation on performance and accuracy if you use this strategy.

In [13]:
print("***** Comparing results with/without paddings *****")
for i in range(2):
    print('Output {} are close:'.format(i), numpy.allclose(opt_outputs[i], ort_outputs[i][:,:len(opt_outputs[i][0])], rtol=1e-03, atol=1e-03))

***** Comparing results with/without paddings *****
Output 0 are close: True
Output 1 are close: True


## 5. Offline Optimization and Test Tools

It is recommended to download the [OnnxRuntime Python Tools for BERT](https://github.com/microsoft/onnxruntime/tree/master/onnxruntime/python/tools/bert), and try them on the exported ONNX models. It could help verify whether the model is fully optimized, and get performance test results.

### Download OnnxRuntime Python Tools for Bert
You may copy the whole [directory](https://github.com/microsoft/onnxruntime/tree/master/onnxruntime/python/tools/bert) to a sub-directory named bert_scripts for this notebook. The list of script files might need update if import error happens when you run some script.

In [14]:
import os
import wget

url_prfix = "https://raw.githubusercontent.com/microsoft/onnxruntime/master/onnxruntime/python/tools/bert/"
script_files = ['bert_perf_test.py', 'bert_test_data.py', 'compare_bert_results.py', 'BertOnnxModel.py', 'BertOnnxModelKeras.py', 'BertOnnxModelTF.py', 'Gpt2OnnxModel.py', 'OnnxModel.py', 'bert_model_optimization.py', 'MachineInfo.py']

script_dir = './bert_scripts'
if not os.path.exists(script_dir):
    os.makedirs(script_dir)

for filename in script_files:
    target_file = os.path.join(script_dir, filename)
    if enable_overwrite and os.path.exists(target_file):
        os.remove(target_file)
    if not os.path.exists(target_file):
        wget.download(url_prfix + filename, target_file)
        print("Downloaded", filename)

100% [..............................................................................] 15310 / 15310Downloaded bert_perf_test.py
100% [................................................................................] 9571 / 9571Downloaded bert_test_data.py
100% [................................................................................] 7272 / 7272Downloaded compare_bert_results.py
100% [..............................................................................] 44905 / 44905Downloaded BertOnnxModel.py
100% [..............................................................................] 21565 / 21565Downloaded BertOnnxModelKeras.py
100% [..............................................................................] 26114 / 26114Downloaded BertOnnxModelTF.py
100% [..............................................................................] 22773 / 22773Downloaded OnnxModel.py
100% [................................................................................] 7795 / 7795

### BERT Optimization Script

Sometime, some optimization of OnnxRuntime cannot be applied to a Bert model due to different reasons:
* A new subgraph pattern is exported, which is not covered by the onnxruntime version users are using. For example, Gelu from PyTorch 1.4 is not fused by OnnxRuntime 1.1.2 (Note: it is covered in OnnxRuntime v1.2.0).
* The exported model uses dynamic axis. That impacts shape inference. Without enough shape information, some optimization cannot be applied due to the constraint on the input shape.
* Some optimization are not supported by OnnxRuntime, but it is feasible in offline script. Like changing input tensor type from int64 to int32 to avoid extra Cast nodes, or converting model to float16 to achieve better performance in V100 or T4 GPU.

We have python script **bert_model_optimization.py**, which is flexible in graph pattern matching and model conversions to tackle these problems.

In below example, we can see that the tool provide an extra optimization - SkipLayerNormalization and bias (Add) are not fused in OnnxRuntime due to shape inference.

The tool will tell whether a model is fully optimized or not. If not, that means you might need change the script to handle some new subgraph patern.

#### Float32 Model
Let us optimize the ONNX model using the script. The first example will output model with float32 to store weights. This is the choice for most GPUs without Tensor Core.

If your GPU (like V100 or T4) has Tensor Core, jump to [Float16 Model](#6.-Model-Optimization-with-Float16) section since that will give you better performance than Float32 model.

In [15]:
GPU_OPTION = '--gpu_only' if use_gpu else ''
optimized_fp32_model_path = './onnx/bert-base-cased-squad_opt_{}_fp32.onnx'.format('gpu' if use_gpu else 'cpu')
%run ./bert_scripts/bert_model_optimization.py --input $export_model_path --output $optimized_fp32_model_path $GPU_OPTION --input_int32

bert_model_optimization.py: Save optimized model by onnxruntime to ./onnx\bert-base-cased-squad_opset11_ort_gpu.onnx
bert_model_optimization.py: Use OnnxRuntime to optimize and save the optimized model to ./onnx\bert-base-cased-squad_opset11_ort_gpu.onnx
    BertOnnxModel.py: Fused LayerNormalization count: 0
    BertOnnxModel.py: Fused Reshape count:0
    BertOnnxModel.py: Fused SkipLayerNormalization count: 24
    BertOnnxModel.py: Fused Attention count:0
    BertOnnxModel.py: skip embed layer fusion since mask input is not found
    BertOnnxModel.py: Fused SkipLayerNormalization with Bias count:24
    BertOnnxModel.py: opset verion: 11
        OnnxModel.py: Output model to ./onnx/bert-base-cased-squad_opt_gpu_fp32.onnx
    BertOnnxModel.py: EmbedLayer=1, Attention=12, Gelu=12, LayerNormalization=24, Succesful=True
bert_model_optimization.py: The output model is fully optimized.


#### Optimized Graph
We can open the optimized model using [Netron](https://github.com/lutzroeder/netron) to visualize.

The graph is like the following:
<img src='images/optimized_bert_gpu.png'>

Sometime, optimized graph is slightly different. For example, FastGelu is replaced by BiasGelu for CPU inference; When the option --input_int32 is used, Cast nodes for inputs are removed.

In [16]:
import netron

# change it to True if want to view the optimized model in browser
enable_netron = False
if enable_netron:
    # If you encounter error "access a socket in a way forbidden by its access permissions", install Netron as standalone application instead.
    netron.start(optimized_fp32_model_path)

### Performance Test Tool

The following will create 1000 random inputs of batch_size 1 and sequence length 128, then measure the average latency and throughput numbers.

Note that the test uses fixed sequence length. If you use [dynamic sequence length](#Inference-with-Actual-Sequence-Length), actual performance depends on the distribution of sequence length.

Note that this tool measures performance of inference using OnnxRuntime Python API. 

In [17]:
GPU_OPTION = '--use_gpu' if use_gpu else ''

%run ./bert_scripts/bert_perf_test.py --model $optimized_fp32_model_path --batch_size 1 --sequence_length 128 --samples 1000 --test_times 1 --inclusive --all $GPU_OPTION

Generating 1000 samples for batch_size=1 sequence_length=128
Extra latency for converting inputs to contiguous: 0.01 ms
Test summary is saved to onnx\perf_results_GPU_B1_S128_20200313-152329.txt


Let's load the summary file and take a look. Note that blank value in OMP_NUM_THREADS or OMP_WAIT_POLICY means the environment variable does not exist.

In [18]:
environment import os
import glob     
import pandas
latest_result_file = max(glob.glob("./onnx/perf_results_GPU_B1_S128_*.txt"), key=os.path.getmtime)
result_data = pandas.read_table(latest_result_file, converters={'OMP_NUM_THREADS': str, 'OMP_WAIT_POLICY':str})
print("Float32 model perf results from", latest_result_file)
# Remove some columns that have same values for all rows.
columns_to_remove = ['model', 'graph_optimization_level', 'batch_size', 'sequence_length', 'test_cases', 'test_times', 'use_gpu']
result_data.drop(columns_to_remove, axis=1, inplace=True)
result_data

Float32 model perf results from ./onnx\perf_results_GPU_B1_S128_20200313-152329.txt


,Latency(ms),Latency_P50,Latency_P75,Latency_P90,Latency_P95,Latency_P99,Throughput(QPS),intra_op_num_threads,OMP_NUM_THREADS,OMP_WAIT_POLICY,contiguous,warmup
0,3.43,3.42,3.44,3.46,3.47,3.58,291.59,6,1,ACTIVE,False,True
1,3.43,3.41,3.43,3.45,3.46,3.54,291.58,6,1,PASSIVE,False,True
2,3.43,3.41,3.43,3.45,3.47,3.81,291.27,6,6,ACTIVE,False,True
3,3.44,3.42,3.44,3.45,3.47,3.57,290.92,0,,,False,True
4,3.44,3.43,3.45,3.46,3.47,3.55,290.46,1,6,ACTIVE,True,True
5,3.44,3.43,3.45,3.46,3.48,3.94,290.31,1,6,PASSIVE,False,True
6,3.45,3.43,3.45,3.47,3.49,3.66,290.23,6,1,ACTIVE,True,True
7,3.45,3.43,3.44,3.46,3.48,3.56,289.90,1,6,PASSIVE,True,True
8,3.45,3.43,3.45,3.47,3.49,4.07,289.74,6,6,PASSIVE,True,True
9,3.46,3.45,3.46,3.48,3.50,3.58,289.17,1,6,ACTIVE,False,True


From above result, we can see that latency is very close for different settings. The default setting (intra_op_num_threads=0, OMP_NUM_THREADS and OMP_WAIT_POLICY does not exist) performs almost on par with the best setting. 

### Model Results Comparison Tool

When a BERT model is optimized, some approximation is used in calculation. If your BERT model has three inputs, a script compare_bert_results.py can be used to do a quick verification. The tool will generate some fake input data, and compare the inference outputs of the original and optimized models. If outputs are all close, it is safe to use the optimized model.

For GPU inference, the absolute or relative difference is larger than those numbers of CPU inference. Note that slight difference in output will not impact final result. We did end-to-end evaluation using SQuAD data set using a fine-tuned squad model, and F1 score is almost the same before/after optimization.

In [19]:
%run ./bert_scripts/compare_bert_results.py --baseline_model $export_model_path --optimized_model $optimized_fp32_model_path --batch_size 1 --sequence_length 128 --samples 100 --rtol 0.01 --atol 0.01 $GPU_OPTION

1 out of 100 results not passed for thresholds (rtol=0.01, atol=0.01).
maximum absolute difference=0.033498868346214294
maximum relative difference=20.89048957824707


## 6. Model Optimization with Float16

The bert_model_optimization.py script have an option **--float16** to convert model to use float16 to store weights. After the conversion, it could be faster to run in GPU with tensor cores like V100 or T4.

Let's run tools to measure the performance on V100. The results show significant performance improvement: latency is about 3.4 ms for float32 model, and 1.8 ms for float16 model.

In [20]:
GPU_OPTION = '--gpu_only' if use_gpu else ''
optimized_fp16_model_path = './onnx/bert-base-cased-squad_opt_{}_fp16.onnx'.format('gpu' if use_gpu else 'cpu')
%run  ./bert_scripts/bert_model_optimization.py --input $export_model_path --output $optimized_fp16_model_path $GPU_OPTION --float16 --input_int32

bert_model_optimization.py: Save optimized model by onnxruntime to ./onnx\bert-base-cased-squad_opset11_ort_gpu.onnx
bert_model_optimization.py: Use OnnxRuntime to optimize and save the optimized model to ./onnx\bert-base-cased-squad_opset11_ort_gpu.onnx
    BertOnnxModel.py: Fused LayerNormalization count: 0
    BertOnnxModel.py: Fused Reshape count:0
    BertOnnxModel.py: Fused SkipLayerNormalization count: 24
    BertOnnxModel.py: Fused Attention count:0
    BertOnnxModel.py: skip embed layer fusion since mask input is not found
    BertOnnxModel.py: Fused SkipLayerNormalization with Bias count:24
    BertOnnxModel.py: opset verion: 11
        OnnxModel.py: Output model to ./onnx/bert-base-cased-squad_opt_gpu_fp16.onnx
    BertOnnxModel.py: EmbedLayer=1, Attention=12, Gelu=12, LayerNormalization=24, Succesful=True
bert_model_optimization.py: The output model is fully optimized.


In [21]:
GPU_OPTION = '--use_gpu' if use_gpu else ''
%run ./bert_scripts/bert_perf_test.py --model $optimized_fp16_model_path --batch_size 1 --sequence_length 128 --samples 1000 --test_times 1 --inclusive --all $GPU_OPTION

Generating 1000 samples for batch_size=1 sequence_length=128
Extra latency for converting inputs to contiguous: 0.00 ms
Test summary is saved to onnx\perf_results_GPU_B1_S128_20200313-152527.txt


In [22]:
import os
import glob     
import pandas
latest_result_file = max(glob.glob("./onnx/perf_results_GPU_B1_S128_*.txt"), key=os.path.getmtime)
result_data = pandas.read_table(latest_result_file, converters={'OMP_NUM_THREADS': str, 'OMP_WAIT_POLICY':str})
print("Float32 model perf results from", latest_result_file)
# Remove some columns that have same values for all rows.
columns_to_remove = ['model', 'graph_optimization_level', 'batch_size', 'sequence_length', 'test_cases', 'test_times', 'use_gpu']
result_data.drop(columns_to_remove, axis=1, inplace=True)
result_data

Float32 model perf results from ./onnx\perf_results_GPU_B1_S128_20200313-152527.txt


,Latency(ms),Latency_P50,Latency_P75,Latency_P90,Latency_P95,Latency_P99,Throughput(QPS),intra_op_num_threads,OMP_NUM_THREADS,OMP_WAIT_POLICY,contiguous,warmup
0,1.84,1.83,1.83,1.85,1.87,2.00,544.94,6,1,PASSIVE,False,True
1,1.85,1.82,1.83,1.85,1.88,2.11,541.89,0,,,False,True
2,1.85,1.83,1.84,1.87,1.90,2.12,541.84,6,1,ACTIVE,False,True
3,1.85,1.84,1.85,1.87,1.89,2.12,541.64,1,6,ACTIVE,True,True
4,1.85,1.83,1.84,1.86,1.89,2.09,541.61,0,,,True,True
5,1.85,1.83,1.84,1.86,1.90,2.07,541.00,6,6,ACTIVE,False,True
6,1.85,1.84,1.84,1.86,1.89,2.18,540.81,1,6,PASSIVE,True,True
7,1.85,1.83,1.84,1.86,1.89,2.08,540.38,6,1,ACTIVE,True,True
8,1.85,1.83,1.84,1.85,1.88,2.16,540.01,1,6,ACTIVE,False,True
9,1.86,1.83,1.84,1.87,1.90,2.14,538.67,1,6,PASSIVE,False,True


### Throughput Tuning

Some application need best throughput under some constraint on latency. This can be done by testing performance of different batch sizes. The tool could help on this.

Here is an example that check the performance of multiple batch sizes (1, 2, 4, 8, 16, 32 and 64) using default settings.

In [23]:
GPU_OPTION = '--use_gpu' if use_gpu else ''
%run ./bert_scripts/bert_perf_test.py --model $optimized_fp16_model_path --batch_size 1 2 4 8 16 32 64 --sequence_length 128 --samples 1000 --test_times 1 --inclusive $GPU_OPTION

Generating 1000 samples for batch_size=32 sequence_length=128
Generating 1000 samples for batch_size=1 sequence_length=128
Generating 1000 samples for batch_size=2 sequence_length=128
Generating 1000 samples for batch_size=64 sequence_length=128
Generating 1000 samples for batch_size=4 sequence_length=128
Generating 1000 samples for batch_size=8 sequence_length=128
Generating 1000 samples for batch_size=16 sequence_length=128
Test summary is saved to onnx\perf_results_GPU_B1-2-4-8-16-32-64_S128_20200313-153014.txt


In [24]:
import os
import glob     
import pandas
latest_result_file = max(glob.glob("./onnx/perf_results_*.txt"), key=os.path.getmtime)
result_data = pandas.read_table(latest_result_file, converters={'OMP_NUM_THREADS': str, 'OMP_WAIT_POLICY':str})
print("Float16 model summary from", latest_result_file)
columns_to_remove = ['model', 'graph_optimization_level', 'test_cases', 'test_times', 'use_gpu', 'warmup', 'sequence_length']

# Set it True to see all rows. Here we only see one setting of intra_op_num_threads==0 and no OMP environment variables
show_all_rows = False
if not show_all_rows:
    result_data = result_data.loc[result_data['intra_op_num_threads']==0]
    columns_to_remove.extend(['intra_op_num_threads', 'OMP_NUM_THREADS', 'OMP_WAIT_POLICY', 'contiguous'])
result_data.drop(columns_to_remove, axis=1, inplace=True)
result_data

Float16 model summary from ./onnx\perf_results_GPU_B1-2-4-8-16-32-64_S128_20200313-153014.txt


,Latency(ms),Latency_P50,Latency_P75,Latency_P90,Latency_P95,Latency_P99,Throughput(QPS),batch_size
0,1.84,1.82,1.84,1.85,1.87,2.09,544.95,1
3,2.19,2.18,2.19,2.20,2.22,2.40,914.56,2
7,2.89,2.87,2.89,2.90,2.92,3.04,1383.38,4
11,4.79,4.76,4.78,4.80,4.84,5.19,1671.42,8
12,8.16,8.15,8.17,8.20,8.22,8.35,1960.21,16
15,14.69,14.66,14.70,14.75,14.80,14.97,2178.49,32
18,28.43,28.42,28.50,28.61,28.68,28.78,2251.36,64


## 7. Additional Info

Note that running Jupyter Notebook has slight impact on performance result since Jupyter Notebook is using system resources like CPU etc. You can close Jupyter Notebook and other applications, then run the performance test in a console to get more accurate performance numbers.

[OnnxRuntime C API](https://github.com/microsoft/onnxruntime/blob/master/docs/C_API.md) could get slightly better performance than python API. If you use C API in inference, you can use OnnxRuntime_Perf_Test.exe built from source to measure performance instead.

Here is the machine configuration that generated the above results. You might get slower or faster result according to your hardware.

In [25]:
%run ./bert_scripts/MachineInfo.py --silent

{
  "gpu": {
    "driver_version": "419.69",
    "devices": [
      {
        "memory_total": 17048272896,
        "memory_available": 12573147136,
        "name": "Tesla V100-PCIE-16GB"
      }
    ]
  },
  "cpu": {
    "brand": "Intel(R) Xeon(R) CPU E5-2690 v4 @ 2.60GHz",
    "cores": 6,
    "logical_cores": 6,
    "hz": "2.5940 GHz",
    "l2_cache": "64",
    "l3_cache": "0 KB",
    "processor": "Intel64 Family 6 Model 79 Stepping 1, GenuineIntel"
  },
  "memory": {
    "total": 120258613248,
    "available": 104351649792
  },
  "python": "3.6.10.final.0 (64 bit)",
  "os": "Windows-10-10.0.14393-SP0",
  "onnxruntime": {
    "version": "1.2.0",
    "support_gpu": true
  },
  "pytorch": {
    "version": "1.4.0",
    "support_gpu": true
  },
  "tensorflow": null
}
